## Policy Gradient Algorithms: REINFORCE

Теорема о градиенте стратегии связывает градиент целевой функции  и градиент самой стратегии:

$$J(\theta) = \mathbb{E}_\pi [\nabla_\theta \ln \pi_\theta(a \vert s) Q^\pi(s, a)]$$
Если использовать метод Монте-Карло в качестве несмещенной оценки $Q^\pi(s, a)$ отдачу $R_t$, то тогда происходит переход к алгоритму REINFORCE и обновление весов будет осуществляться по правилу:

$$J(\theta) = [R_t \nabla_\theta \ln \pi_\theta(A_t \vert S_t)]$$

<img src="reinforce.png">

In [ ]:
# импортируем необходмые библиотеки

import gym
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical

### Задание 1. Заполните пропуски в реализации алгоритма REINFORCE

In [ ]:
class Policy(nn.Module):
    def __init__(self):
        super(Policy, self).__init__()
        # создаем сеть с двумя полносвязными слоями и размером 4 (state input size)-> 128 -> 2 (actions)
        self.fc1 = nn.Linear(4, 128)
        self.fc2 = nn.Linear(128, 2)
        # выбираем Adam optimizer, с lr=0.0002
        self.optimizer = optim.Adam(self.parameters(), lr=0.0002)

    def forward(self, x):
        # как данные будут идти по графу вычислений
        x = F.relu(self.fc1(x))
        x = F.softmax(self.fc2(x), dim=0)
        return x

    def train_net(self, rollout, gamma):
        """
        функция для обучения нашей сети
        :param self:
        :param rollout: [(r, log(pi(s)))]
        :gamma: дисконтирующий множитель
        :return:
        """
        R = 0
        # идем по списку в обратном порядке
        for r, log_prob in rollout[::-1]:
            # считаем суммарное дисконтированное вознаграждение
            #~~~~~~~~~~ Решение ~~~~~~~~~~~~~~~
            
             
            R = r + gamma * R
            
            #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
            

            # считаем значение loss функции
            #~~~~~~~~~~ Решение ~~~~~~~~~~~~~~~
            
             
            loss = -log_prob * R
            
            #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
            

            # обнуляем значения градиента, которые могли накопиться
            self.optimizer.zero_grad()
            # считаем сумму градиентов тензора loss
            loss.backward()
            # выполняем шаг оптимизации
            self.optimizer.step()

Посмотрим на нашу модель и разберем ее методы:

In [ ]:
# создаем модель
e_pi = Policy()

# создаем окружение
e_env = gym.make('CartPole-v1')

# получаем состояние
state = e_env.reset()

# получаем распределение стратегии для состояния
prob = e_pi(torch.from_numpy(state).float())
print("pi("+str(state)+ ") =", prob)

# сэмплируем дейстие, согласно полученному распределению
s_action = Categorical(prob).sample().item()
print("sampled action:", s_action)

In [ ]:
def run():
    env = gym.make('CartPole-v1')
    pi = Policy()
    score = 0.0
    print_interval = 20
    s = env.reset()
    
    for n_epi in range(10000):
        s = env.reset()
        rollout = []
        for t in range(501):  # CartPole-v1 будем рассматривать только первые 500 шагов в окружении
            # получаем распределение стратегии
            #~~~~~~~~~~ Решение ~~~~~~~~~~~~~~~
            
             
            prob = pi(torch.from_numpy(s).float())
            
            #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
            
            
            # сэмплируем действие
            #~~~~~~~~~~ Решение ~~~~~~~~~~~~~~~
            
             
            m = Categorical(prob)
            a = m.sample().item()
            
            #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
            
            
            # делаем шаг в окружении
            s_prime, r, done, info = env.step(a)
            
            # сохраняем информацию текущего шага
            rollout.append((r, torch.log(prob[a])))
            
            s = s_prime
            
            # считаем результат
            score += r
            
            if done:
                break
        
        # обновляем стратегию
        pi.train_net(rollout, 0.98)
        
        # печатаем результаты
        if n_epi % print_interval == 0 and n_epi != 0:
            print("# of episode :{}, avg score : {}".format(n_epi, score / print_interval))
            score = 0.0
    env.close()

In [ ]:
run()

### Задание 2. Дополните код, для постройки графика сходимости. 

Какие выводы можно сделать из этого графика?

### Дополнительные материалы
https://lilianweng.github.io/lil-log/2018/04/08/policy-gradient-algorithms.html